# Punto 1
Inicialice Spark y cargue los datos del archivo "ratings.json" a una RDD llamado "ratings". Usando el archivo metadata.json cree un diccionario llamado "titulos", como variable broadcast de spark, que contenga como "key" el id del libro (id_item) y como "valor" una lista con los siguientes elementos:
El titulo (title), los autores (authors), y el año

In [1]:
! pip install findspark pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark import SparkConf, SparkContext
import findspark

findspark.init()
conf = SparkConf().setMaster("local[*]").setAppName("Seguimiento2")
sc = SparkContext(conf = conf)

In [10]:
#ojo!! BRoadcast solo como diccionario
import json

def built_metadata():
  dictio = {}
  with open("/content/drive/MyDrive/Semestre 9/Arquitectura BigData/book-data/metadata.json") as file:
    for line in file.readlines():
      line = json.loads(line)
      dictio[str(line["item_id"])] = {
          'title' : line['title'],
          'year' :  line['year'],
          'authors': line['authors']
      }
  return dictio

#crear y transmitir el diccionario como variable broadcast
titulos = sc.broadcast(built_metadata())

#visualizar el br
titulos.value

{'16827462': {'title': 'The Fault in Our Stars',
  'year': 2012,
  'authors': 'John Green'},
 '2792775': {'title': 'The Hunger Games (The Hunger Games, #1)',
  'year': 2008,
  'authors': 'Suzanne Collins'},
 '8812783': {'title': 'Mockingjay (The Hunger Games, #3)',
  'year': 2010,
  'authors': 'Suzanne Collins'},
 '41107568': {'title': 'The Girl on the Train',
  'year': 2015,
  'authors': 'Paula Hawkins'},
 '6171458': {'title': 'Catching Fire (The Hunger Games, #2)',
  'year': 2009,
  'authors': 'Suzanne Collins'},
 '878368': {'title': 'The Book Thief', 'year': 0, 'authors': 'Markus Zusak'},
 '15732562': {'title': 'Fifty Shades of Grey (Fifty Shades, #1)',
  'year': 2011,
  'authors': 'E.L. James'},
 '13155899': {'title': 'Divergent (Divergent, #1)',
  'year': 2012,
  'authors': 'Veronica Roth'},
 '3212258': {'title': 'Twilight (Twilight, #1)',
  'year': 2006,
  'authors': 'Stephenie Meyer'},
 '17225055': {'title': 'Eleanor & Park',
  'year': 2013,
  'authors': 'Rainbow Rowell'},
 '155

# Punto 2
Liste el (los) nombre(s) de(l) autor(es) del libro y el nombre del libro con mejor rating, en caso de ser más de un libro listelos todos. Haga lo mismo para el libro con peor rating.

In [15]:
books = sc.textFile('/content/drive/MyDrive/Semestre 9/Arquitectura BigData/book-data/ratings.json')

def parseline(x):
  line = json.loads(x)
  return (str(line['item_id']), float(line['rating']))

books.map(parseline).take(5)

[('41335427', 5.0),
 ('41335427', 3.0),
 ('41335427', 5.0),
 ('41335427', 5.0),
 ('41335427', 5.0)]

In [26]:
books = books.map(parseline) #se deja por fuera para ahorrar proceso en el pto 4

books.mapValues(lambda x: (x,1))\
     .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
     .mapValues(lambda x: round(x[0] / x[1],2))\
     .sortBy(lambda x: -x[1])\
     .map(lambda x: (titulos.value[x[0]]['title'], x[1]))\
     .take(5)

[('Lovely Trigger (Tristan & Danika, #3)', 4.83),
 ('The Gravity of Us (Elements, #4)', 4.82),
 ('March: Book Three (March, #3)', 4.8),
 ('Harry Potter Boxset (Harry Potter, #1-7)', 4.79),
 ('Long Way Down (Calloway Sisters, #4; Addicted, #3.2)', 4.79)]

# Punto 4
Cree un ENDPOINT que se llame “book_by_author” que acepte un parámetro llamado “author” y devuelva una lista con los nombres, el año y la  calificación de los libros del autor del parámetro. Tenga presente que puede existir un libro con más de un autor, si uno de esos autores coincide con el parámetro este libro debería entrar en la lista. También considere hacer la comparación interna (dentro del
ENDPOINT) de los nombres en minúscula y formateando el espacio en blanco con un “+” ejemplo si el autor fuese John Green, el parámetro debe de pasarse como “…?author=John+Green”, el nombre
puede estar en mayúsculas, minúsculas o capitalizado como en este ejemplo

In [20]:
#ngrok
! pip install pyngrok
! ngrok authtoken 2oILEUkg8FP4qNHVEzyLVbnEU0M_6e3GutVah3589eqGbuxLQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [32]:
from flask import Flask, request
from pyngrok import ngrok

app = Flask('Book Ratings')

@app.route('/')
def index():
  return '<h2>Bienvenido a la biblioteca</h2>'

@app.route('/book_by_author')
def get_book_by_author():
  author = request.args.get('autor') # ejemplo book_by_autor?autor=Felix
  #ano = request.args.get('ano') /book_by_author?autor=John%20Green&ano=1999 así se hace para filtrar por 2
  # return results
  results = books.filter(lambda x: titulos.value[x[0]]['authors'] == author)\
                 .mapValues(lambda x: (x,1))\
                 .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
                 .mapValues(lambda x: round(x[0] / x[1],2))\
                 .sortBy(lambda x: -x[1])\
                 .map(lambda x: (titulos.value[x[0]]['title'], x[1]))\
                 .take(25)

  # results = books.mapValues(lambda x: (x,1))\
  #                .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
  #                .mapValues(lambda x: round(x[0] / x[1],2))\
  #                .sortBy(lambda x: -x[1])\
  #                .map(lambda x: (titulos.value[x[0]]['title'], x[1]))\
  #                .take(5)
  # return results
  return results #/book_by_author?autor=John%20Green

tunnel_flask = ngrok.connect(5000) #Flask corre en el 5000 por defecto
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

tunnel_spark = ngrok.connect(4040) #Spark corre en el 4040 por defecto
print(f'URL ngrok for Spark: {tunnel_spark.public_url}')

app.run()

URL ngrok for Flask: https://6c05-35-190-132-245.ngrok-free.app
URL ngrok for Spark: https://7d58-35-190-132-245.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 13:49:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 13:49:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 13:50:23] "GET /book_by_author?autor=John%20Green HTTP/1.1" 200 -
